In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3" 

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", torch_dtype = torch.float16)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
device = 'cuda'

model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [ ]:
instruction = "You are an intelligent assistant trained to classify customer queries into predefined intent categories. Your task is to read a customer's query and assign it to the most appropriate intent category from the following list of intent categories:\n1.activate_my_card\n2.age_limit\n3.apple_pay_or_google_pay\n4.atm_support\n5.automatic_top_up\n6.balance_not_updated_after_bank_transfer\n7.balance_not_updated_after_cheque_or_cash_deposit\n8.beneficiary_not_allowed\n9.cancel_transfer\n10.card_about_to_expire\n11.card_acceptance\n12.card_arrival\n13.card_delivery_estimate\n14.card_linking\n15.card_not_working\n16.card_payment_fee_charged\n17.card_payment_not_recognised\n18.card_payment_wrong_exchange_rate\n19.card_swallowed\n20.cash_withdrawal_charge\n21.cash_withdrawal_not_recognised\n22.change_pin\n23.compromised_card\n24.contactless_not_working\n25.country_support\n26.declined_card_payment\n27.declined_cash_withdrawal\n28.declined_transfer\n29.direct_debit_payment_not_recognised\n30.disposable_card_limits\n31.edit_personal_details\n32.exchange_charge\n33.exchange_rate\n34.exchange_via_app\n35.extra_charge_on_statement\n36.failed_transfer\n37.fiat_currency_support\n38.get_disposable_virtual_card\n39.get_physical_card\n40.getting_spare_card\n41.getting_virtual_card\n42.lost_or_stolen_card\n43.lost_or_stolen_phone\n44.order_physical_card\n45.passcode_forgotten\n46.pending_card_payment\n47.pending_cash_withdrawal\n48.pending_top_up\n49.pending_transfer\n50.pin_blocked\n51.receiving_money\n52.refund_not_showing_up\n53.request_refund\n54.reverted_card_payment\n55.supported_cards_and_currencies\n56.terminate_account\n57.top_up_by_bank_transfer_charge\n58.top_up_by_card_charge\n59.top_up_by_cash_or_cheque\n60.top_up_failed\n61.top_up_limits\n62.top_up_reverted\n63.topping_up_by_card\n64.transaction_charged_twice\n65.transfer_fee_charged\n66.transfer_into_account\n67.transfer_not_received_by_recipient\n68.transfer_timing\n69.unable_to_verify_identity\n70.verify_my_identity\n71.verify_source_of_funds\n72.verify_top_up\n73.virtual_card_not_working\n74.visa_or_mastercard\n75.why_verify_identity\n76.wrong_amount_of_cash_received\n77.wrong_exchange_rate_for_cash_withdrawal\nPlease analyze the query and return the most suitable category from the list above. If the query does not fit perfectly into one category, select the category that best matches the main topic of the query. Only return intent category text and don't return any other words or numbers.\n"

prompt = instruction + "Query: I'm pretty sure something went wrong with my exchange. I changed Russian Ruble to UK pounds and was charged way too much!\nIntent:"
inputs = tokenizer(prompt, return_tensors='pt').to(device)

In [17]:
MAX_NEW_TOKENS = 50
output = model.generate(**inputs, max_new_tokens = MAX_NEW_TOKENS, pad_token_id=tokenizer.eos_token_id, stop_strings = ["\n","\n\n",'<|eot_id|>'], tokenizer=tokenizer).detach().cpu()

In [18]:
output_str = tokenizer.decode(output.squeeze()[len(inputs.input_ids.squeeze()):])
output_str = output_str.lower().strip()

In [19]:
output_str

'exchange_charge'